# Ingest Historical Summoner Games API to Bronze
This notebook loads the historical summoner games (up to April the 16th 2024) data from the League of Legends API. Cf. detail on the API by Riot Games on: https://developer.riotgames.com/

It requires the list of summoners for which we want to get data as input.

The list of summoners from which you want to get data was loaded in a csv file created for that and manually ingested in the Files section of the Bronze Lakehouse. The csv file contains summoner name (which can change) ,the Player Universally Unique ID (puuid, which cannot change) and the Summoner ID (which is needed for some API calls). The puuid was fetched using the summoner API : https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"summoner-name"

_Note: historical games are not actually used as for most old games, the games detail API does not work._


### 1. General configuration

In [ ]:
# Import libraries
from trident_token_library_wrapper import PyTridentTokenLibrary as tl
import requests as r
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
from pyspark.sql import Row
from datetime import datetime
import time

In [ ]:
# Load secrets from Azure Key Vault
key_vault_name = 'testasa-akv-dev-001'
key_name = 'lol-api-key'
access_token = mssparkutils.credentials.getToken("keyvault")
api_key = tl.get_secret_with_token(f"https://{key_vault_name}.vault.azure.net/", key_name, access_token)

# Define API endpoint and parameters
base_url = 'https://europe.api.riotgames.com/lol/'
batch_size = 100

### 2. Load historical Summoner Games data

In [ ]:
# Read Summoners.csv into a DataFrame
print("Reading summoners csv file into a DataFrame")
df_summoners = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ";").csv("Files/raw_data/Summoners.csv")
print("Finished reading summoners csv file into a DataFrame, found", df_summoners.count(), "summoners")

# Make a list out of the PUUIDs to iterate on PUUIDs
puuid_list = df_summoners.select('PUUID').collect()

In [ ]:
# Make API calls to get games of each summoner and collect responses
print("Starting API calls to get summoner games")

all_games = []
current_date = datetime.now().strftime("%Y-%m-%d")

for puuid_row in puuid_list:
    print(f"Getting data for puuid: {puuid_row[0]} ")
    offset = 0
    puuid = puuid_row[0]
    
    while True:
        # Make API call to fetch game identifiers for the current batch
        print(f"Getting data from games {offset} to {offset + batch_size}")
        params = {'api_key': api_key, 'count': batch_size, 'start': offset}
        response = r.get(f"{base_url}match/v5/matches/by-puuid/{puuid}/ids", params=params)
        
        if response.status_code == 200:
            # Append retrieved game identifiers to the list
            games = response.json()
            for game in games:
                all_games.append({"puuid": puuid, "game": game, "date": current_date})
        else:
            print(f"Failed to fetch game identifiers for summoner {puuid}")
            break
        
        # Check if there are more batches to fetch
        if not games:
            break
        else:
            offset += batch_size
            
    time.sleep(30) # see API limits: https://developer.riotgames.com/docs/portal

In [ ]:
# Define the schema for the Summoner Games
schema = StructType([
    StructField("puuid", StringType(), True),
    StructField("game", StringType(), True),
    StructField("date", StringType(), True)
])

# Create DataFrame from the collected rows and schema
df_summoner_games = spark.createDataFrame(all_games, schema=schema)

# Save file to appropriate bronze folder; replace old file if it already existed
print(f"Writing summoner games data to Bronze Lakehouse Files")
df_summoner_games.coalesce(1).write.mode("overwrite").json('Files/raw_data/summoner_games/historical')